
# AI-Driven Optimization of 1-Day VaR (95%) for UK Banks — Notebook Header

> # 1 Project Abstract. 
> This notebook implements a fully reproducible pipeline to forecast next-day 95% Value-at-Risk (VaR) for a portfolio of major UK bank equities (BARC.L, HSBA.L, LLOY.L, NWG.L, STAN.L). Classical benchmarks (GARCH-t, Historical Simulation, Filtered HS) are compared against a compact **LSTM Quantile Regression** and two **hybrids** (feature-fusion with GARCH volatility; convex ensemble). Exogenous drivers include realised volatility, macro state (BoE policy rate, UK 10-year yield) and **FinBERT** news sentiment (bank-specific and macro policy). Evaluation follows industry back-testing (Kupiec/Christoffersen) and proper quantile scoring (pinball loss), with **SHAP** and permutation importance for explainability. The design emphasises transparency, auditability and operational governance consistent with UK PRA SS1/23.

---
> # 2 Project Statement. 
## Data access & folders

- **Market:** Yahoo Finance (`yfinance`) adjusted close for tickers above; daily portfolio return equals equal-weighted bank basket.  
- **Macro:** FRED series `BOERUKM` (Bank Rate) and `IRLTLT01GBM156N` (10Y yield), business-day aligned via forward-fill; FTSE-100 level for context.  
- **Sentiment:** Kaggle news archives (e.g., *News_Category_Dataset_v3.json*, NYT articles) filtered to UK-bank and macro keywords; **FinBERT** polarity averaged into daily indices.  
- **Drive layout (mounted at `/content/drive`)**  
```

/MyDrive/ai-var/
├─ data/
│   ├─ prices/        # Yahoo CSV cache
│   ├─ macro/         # FRED CSV
│   └─ sentiment/     # Kaggle raw + scored daily series
├─ features/          # engineered feature tables
├─ models/            # saved .pt/.pkl and VaR forecasts
├─ eda/               # figures and summary tables
└─ reports/           # comparison tables for paper

````

---

## Dependencies (tested versions)

- Python ≥ 3.11  
- `pandas` ≥ 2.2, `numpy` ≥ 1.26, `scipy` ≥ 1.11  
- `yfinance` ≥ 0.2.40, `fredapi` ≥ 0.5.2  
- `arch` ≥ 6.3, `statsmodels` ≥ 0.14  
- `torch` ≥ 2.2, `scikit-learn` ≥ 1.4  
- `transformers` ≥ 4.41, `shap` ≥ 0.45  
- `matplotlib` ≥ 3.8, `seaborn` ≥ 0.13 (plots)

> **Print runtime versions (optional):**
> ```python
> import sys, torch, pandas as pd, numpy as np, sklearn, shap, transformers, arch, statsmodels, yfinance
> print("Python", sys.version); print("torch", torch.__version__); print("pandas", pd.__version__)
> print("numpy", np.__version__, "sklearn", sklearn.__version__, "shap", shap.__version__, "transformers", transformers.__version__)
> print("arch", arch.__version__, "statsmodels", statsmodels.__version__, "yfinance", yfinance.__version__)
> ```

---

## How to run (end-to-end)

1. **Environment** — install requirements; mount Google Drive.  
2. **Data download** — fetch bank prices (Yahoo), macro (FRED), and load Kaggle news files into `/data/sentiment/`.  
3. **Sentiment scoring** — run FinBERT to create daily `sent_bank`/`sent_macro` series.  
4. **Feature engineering** — compute returns, `abs_ret`, `rv_30d`, macro z-scores, rolling sentiment (e.g., 7-day MA); save to `/features/uk_features.parquet`.  
5. **EDA** — produce distribution, QQ-plot, correlations, rolling sentiment–volatility diagnostics; figures to `/eda/`.  
6. **Baselines** — fit GARCH-t; roll **HS/FHS** VaR; store daily VaR to `/models/VAR/`.  
7. **LSTM-QR** — scale features (train-only stats), build 60-day sequences, train with early-stopping; save `best_model.pt` and predictions.  
8. **Hybrids** — feature-fusion with GARCH σ (Hybrid-Feat); convex ensemble (Hybrid-Ens).  
9. **Back-testing** — compute breaches, Kupiec/Christoffersen p-values, pinball/mean-excess loss; DM tests for loss differentials.  
10. **XAI** — SHAP bars/beeswarm, permutation importance; export to `/reports/`.  
11. **Persist** — dump artefacts (`.pt`, scalers, config JSON, comparison CSV) under `/models/` and `/reports/`.

> **Reproducibility:** set RNG seeds (NumPy/Torch), fix train/val/test temporal splits, log configuration (window lengths, hyper-parameters, data timestamps) in a `config.json`.

---

## Model governance (SS1/23 alignment)

The notebook retains challenger **benchmarks** alongside AI models, records full **data lineage** and configuration versions, produces **explainability artefacts** (SHAP/perm-importance), and saves **coverage & loss** dashboards for monitoring and change control—all consistent with PRA **SS1/23** principles on documentation, performance monitoring and model change governance.

---

## Ethics & data use

Only public/licensed datasets are used; no personal data are processed. News ingestion respects provider licences; code avoids scraping gated content. Sentiment biases are mitigated via domain filters and robustness checks. Forecasts are for research; not investment advice.

---
````


# 3 PIP Package

In [ ]:
# Install core libraries for data, stats, ML, and NLP
!pip install --quiet yfinance pandas_datareader fredapi arch \
    numpy scipy pandas matplotlib seaborn \
    torch torchvision torcheval \
    transformers sentencepiece \
    shap tqdm


In [ ]:
!pip install --quiet kaggle transformers torch tqdm

In [ ]:
# Core imports and version checks
import sys, platform, datetime, torch, numpy as np, pandas as pd
import matplotlib.pyplot as plt

print("Python version:", sys.version.split()[0])
print("Torch version:", torch.__version__)
print("GPU available:", torch.cuda.is_available())
print("Current time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


In [ ]:
# Data acquisition
import yfinance as yf
from pandas_datareader import data as pdr
from fredapi import Fred

# Econometrics & stats
from arch import arch_model
from scipy import stats

# NLP
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 4 Bank Data Downloading

1. UK Banks Stock

In [ ]:
# Parameters
import datetime, yfinance as yf, pandas as pd, numpy as np

# UK bank tickers on LSE (suffix .L)
tickers = ["HSBA.L", "BARC.L", "LLOY.L", "NWG.L", "STAN.L"]
start_date = "2010-01-01"
end_date   = "2025-06-30"


In [ ]:
# auto_adjust=False ⇒ keep original OHLC + Adj Close
data = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    auto_adjust=False,     # <-- key line
    progress=False         # silence the progress bar
)

# Keep only the 'Adj Close' price for each ticker
# data has a 2-level column index: (ticker, field)
prices = data['Adj Close']          # level-1 slice
prices.columns = prices.columns.get_level_values(0)  # flatten ticker names

# Align to business days & forward-fill
prices = prices.asfreq("B").ffill()

# Compute daily log-returns
returns = np.log(prices / prices.shift(1)).dropna()

print("prices shape:", prices.shape)
print("returns shape:", returns.shape)
prices.head()



In [ ]:
# Save to Drive for persistence
prices.to_csv("/content/drive/MyDrive/ai-var/data/uk_banks_prices.csv")
returns.to_csv("/content/drive/MyDrive/ai-var/data/uk_banks_returns.csv")


2. Macro Data

In [ ]:
# Replace 'YOUR_FRED_KEY' with your real key
import os
os.environ["FRED_API_KEY"] = "9c6d07fa024811d37ff986edd232f7c7"
from fredapi import Fred
fred = Fred()


In [ ]:
# ----------------- 0.  准备 -----------------
import pandas as pd, numpy as np, yfinance as yf, os
from fredapi import Fred

start_date, end_date = "2010-01-01", "2025-06-30"
fred = Fred(api_key=os.getenv("9c6d07fa024811d37ff986edd232f7c7"))

def fetch_fred(id_list):
    for sid in id_list:
        try:
            s = fred.get_series(sid, observation_start=start_date, observation_end=end_date)
            if s is not None and s.dropna().size:
                print(f"  FRED {sid} ✓  ({s.size} obs)")
                return s
        except Exception:
            pass
    return None

def fetch_yahoo(ticker, price_field="Close"):
    try:
        df = yf.download(ticker, start=start_date, end=end_date,
                         auto_adjust=False, progress=False)
        if price_field not in df.columns:
            print(f"  Field '{price_field}' missing for {ticker}")
            return None
        s = df[price_field]
        if s.dropna().size:
            print(f"  Yahoo {ticker} ✓  ({s.size} obs)")
            return s
    except Exception as e:
        print(f"  Yahoo {ticker} failed: {e}")
    return None

# ----------------- 1.  收集宏观序列 -----------------
macro_df = pd.DataFrame()

# (1) BoE Base Rate
macro_df["boe_rate"] = fetch_fred(["BOERUKM"])

# (2) 10Y・2Y 英债收益率 (BoE Par Yield 系列)
macro_df["yield_10y"] = fetch_fred(["IUDMNPY10", "IRLTLT01GBM156N"])
macro_df["yield_2y"]  = fetch_fred(["IUDMNPY2" , "IRTLT02GBM156N"])

# (3) 信用利差（缺少可靠公开源 → 暂时跳过）

# ----------------- 2.  计算利差 -----------------
if {"yield_10y", "yield_2y"}.issubset(macro_df.columns):
    macro_df["yield_spread"] = macro_df["yield_10y"] - macro_df["yield_2y"]
    print("yield_spread created.")
else:
    print("yield_spread skipped.")

# ----------------- 3.  股指与波动率 -----------------
ftse_close = fetch_yahoo("^FTSE", "Close")     # FTSE 100 价格
macro_df["ftse_level"] = ftse_close            # 可能为空，稍后再 dropna

# 30-day realised volatility proxy for FTSE 100
if ftse_close is not None:
    log_ret = np.log(ftse_close / ftse_close.shift(1))
    macro_df["vftse_proxy"] = log_ret.rolling(30).std() * np.sqrt(252)
    print("vftse_proxy (30-day realised vol) created.")

# ----------------- 4.  对齐频率 & 清理 -----------------
macro_df = (macro_df
            .asfreq("B")      # business day
            .ffill()          # 前向填充
            .dropna(axis=1, how="all"))  # 移除全空列

print("Current columns:", macro_df.columns.tolist())
print("Shape:", macro_df.shape)

# 持久化
macro_df.to_csv("/content/drive/MyDrive/ai-var/data/uk_macro_daily.csv")

# 5. Align to full business-day calendar, then ffill
full_biz_index = pd.date_range(start=start_date, end=end_date, freq="B")
macro_df = macro_df.reindex(full_biz_index).ffill()

# Optional: silence downcast warning
macro_df = macro_df.infer_objects(copy=False)

print("After reindex, shape:", macro_df.shape)
print(macro_df.head(), "\n", macro_df.tail())



# 5 Sentiment Data and Finbert Factor

In [ ]:
# Move kaggle.json to ~/.kaggle and set permission
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ai-var/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 确认 Kaggle CLI 能列内容
!kaggle datasets list -s reuters -p 10 | head

In [ ]:
!kaggle datasets list -s titanic -p 5 | head   # 只列前 5 行


In [ ]:
!kaggle datasets download -d rmisra/news-category-dataset -p /content/drive/MyDrive/ai-var/data/sentiment/ --unzip
# BuzzFeed
!kaggle datasets download -d konradb/buzzfeed-news-2018-2023 -p /content/drive/MyDrive/ai-var/data/sentiment/ --unzip
# NYT 2020
!kaggle datasets download -d benjaminawd/new-york-times-articles-comments-2020 -p /content/drive/MyDrive/ai-var/data/sentiment/ --unzip


In [ ]:
# ================================================================
# 1. imports & FinBERT
# ================================================================
import glob, os, pandas as pd, numpy as np, torch, tqdm, datetime as dt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax

DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE  = 48
MAX_LEN     = 64

tok  = AutoTokenizer.from_pretrained("ProsusAI/finbert")
mdl  = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(DEVICE).eval()

def finbert_scores(texts, bs=BATCH_SIZE):
    out=[]
    for i in range(0, len(texts), bs):
        batch = texts[i:i+bs]
        inp = tok(batch, truncation=True, padding=True,
                  max_length=MAX_LEN, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            p = softmax(mdl(**inp).logits, dim=1).cpu().numpy()
        out.extend((p[:,0]-p[:,1]).tolist())   # P(pos)-P(neg)
    return np.array(out)

# ================================================================
# 2. keyword list (已扩充)
# ================================================================
bank_kw  = ["hsbc","barclays","lloyds","natwest","standard chartered",
            "hsba","barc","lloy","nwg","stan",
            "uk bank","lender","loan book","credit loss","profit warning",
            "trading desk","capital ratio"]
macro_kw = ["bank of england","boe","interest rate","inflation","gdp",
            "brexit","uk economy","unemployment","recession","bond yield",
            "gilt","sterling","budget","fiscal policy","tax cut"]

def has_kw(text, kws):
    t = str(text).lower()
    return any(k in t for k in kws)

# ================================================================
# 3. dataset config table  (路径可改)
# ================================================================
DATA_DIR = "/content/drive/MyDrive/ai-var/data/sentiment/"       # <-- 修改为你存放解压文件的目录

datasets_cfg = [
    # pattern (glob),                         date_cands,                     text_cols
    ("**/News_Category_Dataset_v3.json",      ['date'],                       ['headline','short_description']),
    ("**/nyt-articles-2020.csv",              ['pub_date','date'],            ['headline','abstract']),
    ("**/bfn-trending-strip-deduped.tsv",     ['timestamp','date','publish_date'], ['title','description']),
]

def load_any(path, date_cols, text_cols):
    ext = os.path.splitext(path)[1].lower()
    sep = '\t' if ext == '.tsv' else ','
    print("→ loading", os.path.basename(path))
    df = (pd.read_json(path, lines=True) if ext=='.json'
          else pd.read_csv(path, sep=sep, low_memory=False))

    # pick first available date column
    date_col = next((c for c in date_cols if c in df.columns), None)
    if date_col is None:
        raise ValueError(f"{path}: no date col among {date_cols}")
    df['date'] = pd.to_datetime(df[date_col], errors='coerce')
    df = df.dropna(subset=['date'])          # remove NaT rows

    for col in text_cols:
        if col not in df.columns:
            df[col] = ''
    df['text'] = df[text_cols].fillna('').agg(' '.join, axis=1)
    return df[['date','text']]

# ------------------------------------------------
# 3a. load all configured datasets
# ------------------------------------------------
df_list = []
for pattern, dcols, tcols in datasets_cfg:
    for fp in glob.glob(os.path.join(DATA_DIR, pattern), recursive=True):
        try:
            df_list.append(load_any(fp, dcols, tcols))
        except Exception as e:
            print("  skip:", e)

news_df = pd.concat(df_list, ignore_index=True)
print("Total merged rows:", news_df.shape[0])

# ================================================================
# 4. keyword filter
# ================================================================
news_df['bank_flag']  = news_df['text'].apply(lambda x: has_kw(x, bank_kw))
news_df['macro_flag'] = news_df['text'].apply(lambda x: has_kw(x, macro_kw))

bank_df  = news_df[news_df.bank_flag]
macro_df = news_df[news_df.macro_flag]
print("Bank rows:", bank_df.shape[0], "| Macro rows:", macro_df.shape[0])

# ================================================================
# 5. robust daily sentiment function (fix .dt bug)
# ================================================================
def daily_sentiment(sub_df, sent_col, vol_col):
    if sub_df.empty:
        return pd.DataFrame(columns=[sent_col, vol_col])

    sub_df = sub_df.copy()
    sub_df['date'] = pd.to_datetime(sub_df['date'], errors='coerce')
    sub_df = sub_df.dropna(subset=['date'])

    print(f"Scoring {sent_col} — {len(sub_df)} articles")
    sub_df.loc[:, 'score'] = finbert_scores(sub_df['text'].tolist())

    daily = (sub_df
             .groupby(sub_df['date'].dt.date)['score']
             .agg(['mean','count'])
             .rename(columns={'mean': sent_col, 'count': vol_col}))
    return daily

bank_daily  = daily_sentiment(bank_df,  'sent_bank',  'vol_bank')
macro_daily = daily_sentiment(macro_df, 'sent_macro', 'vol_macro')

sentiment = pd.concat([bank_daily, macro_daily], axis=1).sort_index()

# align to business-day index
biz_idx  = pd.date_range(sentiment.index.min(), sentiment.index.max(), freq='B')
sentiment = (sentiment
             .reindex(biz_idx)
             .fillna(0.0)
             .infer_objects(copy=False))

print("Final sentiment shape:", sentiment.shape)

# ================================================================
# 6. save to Google Drive
# ================================================================
from google.colab import drive
drive.mount('/content/drive')

OUT_CSV = "/content/drive/MyDrive/ai-var/data/uk_sentiment_daily.csv"
sentiment.to_csv(OUT_CSV)
print("✅ Sentiment CSV saved to:", OUT_CSV)


# 6 Data Preprocessing and Feature Engineering

In [ ]:
import pandas as pd, numpy as np

RET_PATH = "/content/drive/MyDrive/ai-var/data/uk_banks_returns.csv"
returns_raw = pd.read_csv(RET_PATH, index_col=0, parse_dates=True)

# 把所有列转换成数值 (有空字符串会变 NaN)
returns_raw = returns_raw.apply(pd.to_numeric, errors='coerce')

# ---------- 方案 A: 等权组合 ----------
returns_raw['return'] = returns_raw.mean(axis=1)      # 简单平均

returns_raw.to_csv("/content/drive/MyDrive/ai-var/data/uk_bank_returns.csv")
print("✅ created 'return' column — shape now", returns_raw.shape)


In [ ]:
import pandas as pd, numpy as np
from google.colab import drive
drive.mount('/content/drive')

RET_PATH  = "/content/drive/MyDrive/ai-var/data/uk_bank_returns.csv"
MACRO_PATH= "/content/drive/MyDrive/ai-var/data/uk_macro_daily.csv"
SENT_PATH = "/content/drive/MyDrive/ai-var/data/uk_sentiment_daily.csv"

returns = pd.read_csv(RET_PATH,  index_col=0, parse_dates=True)
macro   = pd.read_csv(MACRO_PATH, index_col=0, parse_dates=True)
sent    = pd.read_csv(SENT_PATH,  index_col=0, parse_dates=True)

# 以 returns 为主轴左连接
df = (returns
      .join([macro, sent], how='left')
      .sort_index())

# 先 forward-fill 宏观，再把情绪 NaN → 0
macro_cols = macro.columns.tolist()
sent_cols  = ['sent_bank','vol_bank','sent_macro','vol_macro']
df[macro_cols] = df[macro_cols].fillna(method='ffill')
df[sent_cols]  = df[sent_cols].fillna(0.0)

print("Merged shape:", df.shape)
print(df.head())


In [ ]:
# 绝对收益、平方收益
df['abs_ret']   = df['return'].abs()
df['ret_sq']    = df['return']**2

# 实现波动率 (30d)
df['rv_30d'] = df['return'].rolling(30).std()

# 滚动均值收益 (5d)
df['ret_mean_5d'] = df['return'].rolling(5).mean()



In [ ]:
# 7 天滚动均值 & z-score
for col in ['sent_bank','sent_macro']:
    df[f'{col}_7d'] = df[col].rolling(7, min_periods=1).mean()

# 新闻量 30 天累计
df['vol_bank_30d']  = df['vol_bank'].rolling(30).sum()
df['vol_macro_30d'] = df['vol_macro'].rolling(30).sum()


In [ ]:
# 一阶差分 / 对数差分
df['boe_rate_diff']  = df['boe_rate'].diff()
df['yield_10y_diff'] = df['yield_10y'].diff()

# 对宏观做 z-score（基于全部样本）
for col in ['boe_rate','yield_10y','boe_rate_diff','yield_10y_diff']:
    df[f'{col}_z'] = (df[col] - df[col].mean()) / df[col].std()


In [ ]:
# 剩余 NaN（因 rolling 导致前几行缺值）→ 丢弃
df = df.dropna()

# Winsorize: 把 return 极端 0.5% 修正到阈值
lo, hi = df['return'].quantile([0.005, 0.995])
df['return'] = df['return'].clip(lo, hi)


In [ ]:
# ======================================================
# 6-6  保存处理完的特征表 & 标准化器
# ======================================================
from sklearn.preprocessing import StandardScaler
import joblib, os

# ------------ 配置 ------------
FEATURES = ['return','abs_ret','rv_30d','ret_mean_5d',
            'sent_bank_7d','sent_macro_7d',
            'vol_bank_30d','vol_macro_30d',
            'boe_rate_z','yield_10y_z','boe_rate_diff_z','yield_10y_diff_z']
TARGET   = 'return'
SAVE_DIR = "/content/drive/MyDrive/ai-var/processed"
os.makedirs(SAVE_DIR, exist_ok=True)

# ------------ 拆分 & 标准化 ------------
split_date = '2022-01-01'
train_df   = df.loc[:split_date].copy()
test_df    = df.loc[split_date:].copy()

scaler = StandardScaler()
train_df[FEATURES] = scaler.fit_transform(train_df[FEATURES])
test_df[FEATURES]  = scaler.transform(test_df[FEATURES])

# ------------ 合并回完整集（方便统一存储） ------------
df_scaled = pd.concat([train_df, test_df]).sort_index()



In [ ]:
# ------------ 保存数据 ------------
parquet_path = f"/content/drive/MyDrive/ai-var/models/uk_features_scaled.parquet"
csv_path     = f"/content/drive/MyDrive/ai-var/models/uk_features_scaled.csv.gz"
scaler_path  = f"/content/drive/MyDrive/ai-var/models/feature_scaler.bin"

df_scaled.to_parquet(parquet_path, compression='snappy')
df_scaled.to_csv(csv_path, compression='gzip')
joblib.dump(scaler, scaler_path)

print("✅ Saved:")
print("  · Parquet :", parquet_path)
print("  · CSV (gz):", csv_path)
print("  · Scaler  :", scaler_path)
print("Shape:", df_scaled.shape, " | Columns:", df_scaled.columns.tolist()[:10], "...")

# 7 EDA

In [ ]:
#7-A️⃣ 数据概览 & 缺失情况
print(df_scaled.info())
print(df_scaled.describe(percentiles=[.01,.05,.5,.95,.99]))

# missing value heatmap
import seaborn as sns, matplotlib.pyplot as plt
sns.heatmap(df_scaled.isna(), cbar=False)
plt.title("Missing value map"); plt.show()


In [ ]:
7-B️⃣ 收益分布特征
import matplotlib.pyplot as plt
import scipy.stats as st
from statsmodels.graphics.gofplots import qqplot

# Histogram + kernel density
df_scaled['return'].plot(kind='hist', bins=80, density=True, alpha=0.6)
df_scaled['return'].plot(kind='kde', linewidth=2)
plt.title("Distribution of daily log return"); plt.show()

# QQ-plot vs Normal
qqplot(df_scaled['return'], line='s')
plt.title("QQ-plot (return vs Normal)"); plt.show()

# Jarque-Bera test for normality
jb_stat, jb_p = st.jarque_bera(df_scaled['return'])
print(f"Jarque-Bera stat={jb_stat:.2f},  p-value={jb_p:.3g}")


In [ ]:
7-C️⃣ 时序可视化：收益、波动、情绪、宏观
fig, ax = plt.subplots(4,1, figsize=(12,10), sharex=True)
df_scaled['return'].plot(ax=ax[0]); ax[0].set_title("Daily return")
df_scaled['rv_30d'].plot(ax=ax[1]); ax[1].set_title("30-day realised vol")
df_scaled['sent_bank_7d'].plot(ax=ax[2]); ax[2].set_title("Bank sentiment 7-day MA")
df_scaled['boe_rate_z'].plot(ax=ax[3]); ax[3].set_title("BOE rate (z-score)")
plt.tight_layout(); plt.show()


In [ ]:
7-D️⃣ 相关性矩阵 & 滚动相关
import seaborn as sns

corr = df_scaled[['return','abs_ret','rv_30d',
                  'sent_bank_7d','sent_macro_7d',
                  'boe_rate_z','yield_10y_z']].corr()
plt.figure(figsize=(7,5))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
plt.title("Pearson correlation matrix"); plt.show()

# 180-day rolling correlation between sentiment & volatility
roll = df_scaled[['sent_bank_7d','rv_30d']].rolling(180).corr().dropna()
roll.loc[pd.IndexSlice[:,'sent_bank_7d'],'rv_30d'].droplevel(1).plot(figsize=(9,3))
plt.axhline(0, color='k', ls='--'); plt.title("Rolling corr: sentiment vs vol"); plt.show()


In [ ]:
7-E️⃣ 情绪滞后与未来波动
# 1 日后绝对收益作为目标
df_tmp = pd.DataFrame({
    'sent_bank_7d' : df_scaled['sent_bank_7d'],
    'abs_ret_t+1'  : df_scaled['abs_ret'].shift(-1)
}).dropna()

sns.regplot(x='sent_bank_7d', y='abs_ret_t+1', data=df_tmp,
            scatter_kws={'s':10, 'alpha':0.3})
plt.title("Bank sentiment vs next-day abs return"); plt.show()

rho = df_tmp.corr().iloc[0,1]
print("Pearson corr (sentiment vs next-day volatility):", rho)


In [ ]:
7-F️⃣ 平稳性和自相关
from statsmodels.tsa.stattools import adfuller, acf, pacf, q_stat
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

adf_stat, adf_p, *_ = adfuller(df_scaled['return'])
print(f"ADF stat={adf_stat:.2f}  p-value={adf_p:.3g}")

fig, ax = plt.subplots(2,1, figsize=(10,6))
plot_acf(df_scaled['return'], lags=40, ax=ax[0])
plot_pacf(df_scaled['return'], lags=40, ax=ax[1]); plt.show()

# Ljung-Box for up to 10 lags
from statsmodels.stats.diagnostic import acorr_ljungbox
lb = acorr_ljungbox(df_scaled['return'], lags=[10], return_df=True)
print(lb)


In [ ]:
# ==============================================================
# Set directory to save all EDA outputs
# ==============================================================
import os, matplotlib.pyplot as plt, seaborn as sns, numpy as np, pandas as pd
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

EDA_DIR = "/content/drive/MyDrive/ai-var/eda"
os.makedirs(EDA_DIR, exist_ok=True)

# ------------- Helper to save and clear --------------
def save_show(fig, filename):
    path = f"{EDA_DIR}/{filename}"
    fig.savefig(path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print("saved:", path)

# ==============================================================
# 7-A  distribution stats already printed in console
# ==============================================================
# no figure to save here

# ==============================================================
# 7-B  Return distribution histogram & KDE
# ==============================================================
fig, ax = plt.subplots(figsize=(7,4))
sns.histplot(df_scaled['return'], bins=80, stat='density', alpha=0.6, ax=ax)
sns.kdeplot(df_scaled['return'], ax=ax, linewidth=2)
ax.set_title("Return distribution & KDE")
save_show(fig, "return_distribution.png")

# QQ-plot
fig = qqplot(df_scaled['return'], line='s')
plt.title("QQ-plot: return vs normal")
save_show(fig, "return_qqplot.png")

# ==============================================================
# 7-C  Time-series plot (returns, vol, sentiment, BOE rate)
# ==============================================================
fig, axes = plt.subplots(4,1, figsize=(12,10), sharex=True)
df_scaled['return'].plot(ax=axes[0]); axes[0].set_title("Daily return")
df_scaled['rv_30d'].plot(ax=axes[1]); axes[1].set_title("30-day realised vol")
df_scaled['sent_bank_7d'].plot(ax=axes[2]); axes[2].set_title("Bank sentiment 7-day MA")
df_scaled['boe_rate_z'].plot(ax=axes[3]); axes[3].set_title("BOE rate (z-score)")
plt.tight_layout()
save_show(fig, "timeseries_overview.png")

# ==============================================================
# 7-D  Correlation heatmap & rolling correlation
# ==============================================================
corr = df_scaled[['return','abs_ret','rv_30d','sent_bank_7d','sent_macro_7d',
                  'boe_rate_z','yield_10y_z']].corr()
fig, ax = plt.subplots(figsize=(7,5))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", ax=ax)
ax.set_title("Correlation matrix")
save_show(fig, "corr_heatmap.png")
corr.to_csv(f"{EDA_DIR}/correlation_matrix.csv")

# rolling correlation
roll = df_scaled[['sent_bank_7d','rv_30d']].rolling(180).corr().dropna()
series = roll.loc[pd.IndexSlice[:,'sent_bank_7d'],'rv_30d'].droplevel(1)
fig, ax = plt.subplots(figsize=(9,3))
series.plot(ax=ax); ax.axhline(0, ls='--', c='k')
ax.set_title("180-day rolling corr: sentiment vs volatility")
save_show(fig, "rolling_corr_sent_vs_vol.png")

# ==============================================================
# 7-E  Sentiment vs next-day abs return
# ==============================================================
df_tmp = pd.DataFrame({
    'sent_bank_7d': df_scaled['sent_bank_7d'],
    'abs_ret_t+1' : df_scaled['abs_ret'].shift(-1)
}).dropna()
fig, ax = plt.subplots(figsize=(6,4))
sns.regplot(x='sent_bank_7d', y='abs_ret_t+1',
            data=df_tmp, scatter_kws={'s':10,'alpha':0.3}, ax=ax)
ax.set_title("Bank sentiment vs next-day abs return")
save_show(fig, "sentiment_vs_nextday_vol.png")
df_tmp.corr().to_csv(f"{EDA_DIR}/sentiment_future_vol_corr.csv")

# ==============================================================
# 7-F  ACF / PACF & QQ stats
# ==============================================================
fig, axes = plt.subplots(2,1, figsize=(10,6))
plot_acf(df_scaled['return'], lags=40, ax=axes[0])
plot_pacf(df_scaled['return'], lags=40, ax=axes[1])
axes[0].set_title("ACF – returns"); axes[1].set_title("PACF – returns")
save_show(fig, "acf_pacf_returns.png")

# ADF test results txt
adf_stat, adf_p, *_ = adfuller(df_scaled['return'])
with open(f"{EDA_DIR}/adf_test.txt", 'w') as f:
    f.write(f"ADF stat = {adf_stat:.4f}\np-value = {adf_p:.6f}\n")
print("saved: adf_test.txt")

print("\n🎉 All EDA figures & files saved in:", EDA_DIR)



# 8 Benchmark VAR Model

In [ ]:
!pip install --quiet arch statsmodels scipy tqdm


In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from arch import arch_model
from scipy import stats
from tqdm.auto import tqdm


In [ ]:
#8-A 数据集划分
ALPHA       = 0.05           # 95% VaR
ROLL_WIN    = 250            # HS / FHS rolling window
SPLIT_DATE  = '2022-01-01'   # 训练 – 测试分割

train_ret = df_scaled.loc[:SPLIT_DATE, 'return'] * 100   # *100 for arch (百分数)
test_ret  = df_scaled.loc[SPLIT_DATE:, 'return'] * 100

In [ ]:
#8-B GARCH(1,1) + Student-t
#8-B-1 拟合
garch = arch_model(train_ret, mean='Zero', vol='GARCH',
                   p=1, q=1, dist='StudentsT').fit(disp="off")
print(garch.summary())


In [ ]:
#8-B-2 滚动预测 VaR
def garch_rolling_var(result, test_series, alpha=ALPHA):
    """
    result : fitted ARCHModelResult (GARCH(1,1)-t)
    test_series : pd.Series of daily returns *100
    returns     : pd.Series of VaR forecasts (positive number)
    """
    params = result.params
    omega  = params['omega']
    alpha1 = params['alpha[1]']
    beta1  = params['beta[1]']
    nu     = params['nu']

    # last in-sample conditional variance & residual
    sigma_prev = result.conditional_volatility.iloc[-1]
    eps_prev   = result.std_resid.iloc[-1] * sigma_prev

    var_list = []

    for ret in tqdm(test_series, desc="GARCH rolling"):
        # one-step-ahead variance forecast
        h_t = omega + alpha1 * eps_prev**2 + beta1 * sigma_prev**2
        sigma_t = np.sqrt(h_t)

        q = stats.t.ppf(alpha, df=nu)          # left-tail quantile (<0)
        var_list.append(-(sigma_t * q) / 100)  # back to raw scale (+)

        # update for next iteration
        eps_prev   = ret                        # because mean=0 in model
        sigma_prev = sigma_t

    return pd.Series(var_list, index=test_series.index, name='VaR_garch')


VaR_garch = garch_rolling_var(garch, test_ret)


In [ ]:
#8-C 滚动历史模拟 (HS)
def hs_var(train_series, test_series, win=ROLL_WIN, alpha=ALPHA):
    var_series = []
    history = train_series.copy()
    for date, ret in tqdm(test_series.items(), desc="HS rolling"):
        var = -np.quantile(history/100, alpha)   # history still *100, rescale back
        var_series.append(var)
        history = pd.concat([history, pd.Series([ret], index=[date])]).iloc[-win:]
    return pd.Series(var_series, index=test_series.index, name='VaR_hs')

VaR_hs = hs_var(train_ret, test_ret)


In [ ]:
#8-D Filtered Historical Simulation (FHS)
# 1) get standardized residuals from fitted GARCH
std_resid_train = garch.std_resid * np.sqrt((garch.params['nu']-2)/garch.params['nu'])

def fhs_var(result, test_series, alpha=ALPHA, win=ROLL_WIN):
    """
    Filtered Historical Simulation (FHS) VaR
    result      : fitted ARCHModelResult (训练期 GARCH(1,1)-t)
    test_series : pd.Series of returns *100
    returns     : pd.Series VaR (正值)
    """
    p = result.params
    omega, a1, b1, nu = p['omega'], p['alpha[1]'], p['beta[1]'], p['nu']

    # 训练期最后的 σ_T 和 ε_T
    sigma_prev = result.conditional_volatility.iloc[-1]
    eps_prev   = result.std_resid.iloc[-1] * sigma_prev

    # 历史标准化残差（t 调整）
    std_resid = result.std_resid * np.sqrt((nu-2)/nu)
    hist_std  = pd.Series(std_resid, index=result.conditional_volatility.index).copy()

    var_list = []

    for date, ret in tqdm(test_series.items(), desc="FHS rolling"):
        # 1-step ahead variance
        h_t = omega + a1 * eps_prev**2 + b1 * sigma_prev**2
        sigma_t = np.sqrt(h_t)

        # VaR = -σ_t * q_α(ε_std)
        q = np.quantile(hist_std, alpha)
        var_list.append(-(sigma_t * q) / 100)

        # 更新残差与 σ_t，维护滚动窗口
        eps_prev   = ret
        sigma_prev = sigma_t
        new_std    = eps_prev / sigma_prev
        hist_std   = pd.concat([hist_std, pd.Series([new_std], index=[date])]).iloc[-win:]

    return pd.Series(var_list, index=test_series.index, name='VaR_fhs')


VaR_fhs = fhs_var(garch, test_ret)


In [ ]:
#8-E 回测与评估
def kupiec_test(returns, var, alpha=ALPHA):
    breaches = (returns < -var*100)  # returns still *100
    n, x = len(breaches), breaches.sum()
    p_hat = x/n
    LR_uc = -2 * ( np.log((1-alpha)**(n-x) * alpha**x) -
                   np.log((1-p_hat)**(n-x) * p_hat**x) )
    p_value = 1 - stats.chi2.cdf(LR_uc, df=1)
    return x, p_value

def christoffersen_test(returns, var, alpha=ALPHA):
    breaches = (returns < -var*100).astype(int)
    N00=N01=N10=N11=0
    for i in range(1,len(breaches)):
        if breaches[i-1]==0 and breaches[i]==0: N00+=1
        if breaches[i-1]==0 and breaches[i]==1: N01+=1
        if breaches[i-1]==1 and breaches[i]==0: N10+=1
        if breaches[i-1]==1 and breaches[i]==1: N11+=1
    pi0 = N01/(N00+N01) if (N00+N01)>0 else 0
    pi1 = N11/(N10+N11) if (N10+N11)>0 else 0
    pi  = (N01+N11)/(N00+N01+N10+N11)
    LR_cc = -2*np.log( ( (1-pi)**(N00+N10) * pi**(N01+N11) ) /
                       ( (1-pi0)**N00 * pi0**N01 * (1-pi1)**N10 * pi1**N11 ) )
    p_value = 1 - stats.chi2.cdf(LR_cc, df=2)
    return p_value

def quantile_loss(returns, var, alpha=ALPHA):
    y = returns/100
    q = -var
    e = y - q
    return np.mean(np.maximum(alpha*e, (alpha-1)*e))

results = {}
for label, series in [('GARCH', VaR_garch), ('HS', VaR_hs), ('FHS', VaR_fhs)]:
    breaches, p_kup = kupiec_test(test_ret, series, ALPHA)
    p_ch  = christoffersen_test(test_ret, series, ALPHA)
    ql    = quantile_loss(test_ret, series, ALPHA)
    results[label] = [breaches, p_kup, p_ch, ql]

res_df = pd.DataFrame(results, index=['#Breaches','Kupiec_p','Christoffersen_p','QLoss'])
print(res_df.T)
res_df.T.to_csv(f"/content/drive/MyDrive/ai-var/models/VAR/var_backtest_stats.csv")


In [ ]:

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,4))

# actual returns (灰色，缩放回原始 0–1 区间)
(test_ret/100).plot(ax=ax, color='grey', alpha=0.45, label='Return')

# VaR curves (正值表示潜在损失)
(-VaR_garch).plot(ax=ax, color='red',   label='GARCH VaR95')
(-VaR_fhs).plot(  ax=ax, color='blue',  label='FHS VaR95', alpha=0.7)
(-VaR_hs ).plot(  ax=ax, color='green', label='HS VaR95',  alpha=0.7)

ax.axhline(0, color='k', linewidth=0.6)
ax.set_title("VaR vs Returns (95% level)")
ax.legend(loc='upper right')
ax.set_ylabel("Loss level (+)  /  Return (-)")

# Save to Drive
os.makedirs(SAVE_DIR, exist_ok=True)
path = f"/content/drive/MyDrive/ai-var/models/VAR/var_vs_returns.png"
plt.savefig(path, dpi=300, bbox_inches='tight')
plt.show()

print("✅ figure saved to:", path)



# 9 LSTM Model

In [ ]:
!pip install --quiet torch torchvision torchaudio tqdm scikit-learn joblib

import torch, torch.nn as nn, numpy as np, pandas as pd, joblib, os
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

DEVICE     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEQ_LEN    = 60          # 序列长度 (60 天)
BATCH      = 256         # 批大小，如显存不足改小
EPOCHS     = 40
ALPHA      = 0.05        # 95% VaR
SPLIT_DATE = '2022-01-01'
DATA_DIR   = "/content/drive/MyDrive/ai-var/models/processed"
MODEL_DIR  = "/content/drive/MyDrive/ai-var/models/LSTM"
os.makedirs(MODEL_DIR, exist_ok=True)


In [ ]:
#9-1 读取数据 & 构造序列
# 9-1-1  load scaled features
df = pd.read_parquet(f"{DATA_DIR}/uk_features_scaled.parquet")

FEATURES = ['return','abs_ret','rv_30d','ret_mean_5d',
            'sent_bank_7d','sent_macro_7d',
            'vol_bank_30d','vol_macro_30d',
            'boe_rate_z','yield_10y_z','boe_rate_diff_z','yield_10y_diff_z']
TARGET   = 'return'

# 9-1-2  split
train_df = df.loc[:SPLIT_DATE]
test_df  = df.loc[SPLIT_DATE:]

# 9-1-3  build sequences ----
def build_xy(df, seq_len=SEQ_LEN):
    X, y = [], []
    for i in range(seq_len, len(df)):
        X.append(df[FEATURES].iloc[i-seq_len:i].values.astype('float32'))
        y.append(df[TARGET].iloc[i])
    return np.stack(X), np.array(y, dtype='float32')

X_train, y_train = build_xy(train_df)
X_test,  y_test  = build_xy(test_df)

print("Train seq:", X_train.shape, " Test seq:", X_test.shape)

# convert to tensors
train_ds = torch.utils.data.TensorDataset(
    torch.tensor(X_train), torch.tensor(y_train).unsqueeze(1))
test_ds  = torch.utils.data.TensorDataset(
    torch.tensor(X_test), torch.tensor(y_test).unsqueeze(1))

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH, shuffle=True)


In [ ]:
#9-2 定义网络 & Pinball Loss
class LSTM_QR(nn.Module):
    def __init__(self, n_feat, hidden=64, layers=1):
        super().__init__()
        self.lstm = nn.LSTM(n_feat, hidden, layers, batch_first=True)
        self.fc   = nn.Linear(hidden, 1)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:,-1])

def quantile_loss(pred, target, q=ALPHA):
    e = target - pred
    return torch.mean(torch.maximum(q*e, (q-1)*e))

model = LSTM_QR(len(FEATURES)).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
#9-3 训练（含 Early-Stopping）
BEST_PATH = f"{MODEL_DIR}/lstm_qr_best.pt"
best_val  = np.inf
patience  = 5
no_improve = 0

for epoch in range(EPOCHS):
    # ---------- training ----------
    model.train()
    running = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        pred = model(xb)
        loss = quantile_loss(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running += loss.item() * len(xb)
    train_loss = running / len(train_ds)

    # ---------- validation ----------
    model.eval()
    with torch.no_grad():
        val_pred = model(torch.tensor(X_test).to(DEVICE))
        val_loss = quantile_loss(val_pred.cpu(), torch.tensor(y_test).unsqueeze(1)).item()

    print(f"Epoch {epoch+1:02d} | train {train_loss:.4f} | val {val_loss:.4f}")

    # ---------- early stopping ----------
    if val_loss < best_val - 1e-4:
        best_val = val_loss
        no_improve = 0
        torch.save(model.state_dict(), BEST_PATH)
        print("  ↳ saved best model")
    else:
        no_improve += 1
        if no_improve >= patience:
            print("Early stopping triggered.")
            break



In [ ]:
#9-4 载入最佳模型 & 预测 VaR
best_model = LSTM_QR(len(FEATURES)).to(DEVICE)
best_model.load_state_dict(torch.load(BEST_PATH))
best_model.eval()

with torch.no_grad():
    pred_test = best_model(torch.tensor(X_test).to(DEVICE)).cpu().numpy().flatten()

# VaR95 = -预测值 (pred 为负收益 quantile)
VaR_lstm = pd.Series(-pred_test, index=test_df.index[SEQ_LEN:], name='VaR_lstm')


In [ ]:
#9-5 回测 & 与基准比较
# ------------------ backtest util (同前) ------------------
def kupiec(returns, var, alpha=ALPHA):
    exc = (returns < -var).astype(int)
    n, x = len(exc), exc.sum()
    p_hat = x/n
    LR_uc = -2*( np.log((1-alpha)**(n-x) * alpha**x) -
                 np.log((1-p_hat)**(n-x) * p_hat**x) )
    p = 1-stats.chi2.cdf(LR_uc,1)
    return x, p

def qloss(returns, var, alpha=ALPHA):
    y = returns
    q = -var
    e = y - q
    return np.mean(np.maximum(alpha*e, (alpha-1)*e))

# align returns
ret_lstm = test_df[TARGET].iloc[SEQ_LEN:] * 100  # back to %
stats_lstm = kupiec(ret_lstm, VaR_lstm*100)
print("LSTM breaches:", stats_lstm[0], "Kupiec p:", stats_lstm[1])
print("LSTM QLoss:", qloss(ret_lstm, VaR_lstm*100))

# ---------- compare with GARCH / FHS ----------
comp = pd.concat([VaR_garch, VaR_fhs, VaR_lstm], axis=1).dropna()
ret_cmp = ret_lstm.reindex(comp.index)

for name, series in comp.items():
    b, p = kupiec(ret_cmp, series*100)
    print(f"{name:10s}  breaches={b:3d}  Kupiec_p={p:.3f}  QLoss={qloss(ret_cmp, series*100):.5f}")

# ---------- plotting ----------
fig, ax = plt.subplots(figsize=(12,4))
(ret_cmp/100).plot(ax=ax, color='grey', alpha=0.4, label='Return')
(-comp['VaR_garch']).plot(ax=ax, color='red',   label='GARCH')
(-comp['VaR_fhs']).plot(  ax=ax, color='blue',  label='FHS', alpha=0.7)
(-comp['VaR_lstm']).plot( ax=ax, color='orange',label='LSTM', alpha=0.8)
ax.axhline(0, color='k', lw=0.5); ax.set_title("VaR95 — LSTM vs Baselines")
ax.legend(); plt.savefig(f"{MODEL_DIR}/var_lstm_vs_baselines.png", dpi=300); plt.show()


# 10 Hybrid GARCH + LSTM

In [ ]:
#步骤 1-a 生成训练期序列
# garch 是训练好 的 ARCHModelResult (含训练期)
sigma_train = garch.conditional_volatility          # len = train_ret
# 滚动递推 σ_t 到测试期
sigma_roll = []
sigma_prev = sigma_train.iloc[-1]
eps_prev   = garch.std_resid.iloc[-1] * sigma_prev
omega, a1, b1 = garch.params[['omega','alpha[1]','beta[1]']]

for ret in test_ret:            # test_ret 已 *100
    h = omega + a1*eps_prev**2 + b1*sigma_prev**2
    sigma = np.sqrt(h)
    sigma_roll.append(sigma)
    eps_prev, sigma_prev = ret, sigma

sigma_test = pd.Series(sigma_roll, index=test_ret.index)

# 合并进 df_scaled （注意 /100 回原点）
df_hybrid = df_scaled.copy()
df_hybrid['garch_sigma'] = pd.concat([sigma_train/100, sigma_test/100])


In [ ]:
#步骤 1-b 重建序列、训练 LSTM-QR
FEATURES_H = FEATURES + ['garch_sigma']          # 在原 Feature 列后追加
SEQ_LEN     = 60

def build_xy_h(df):
    X, y = [], []
    for i in range(SEQ_LEN, len(df)):
        X.append(df[FEATURES_H].iloc[i-SEQ_LEN:i].values.astype('float32'))
        y.append(df[TARGET].iloc[i])
    return np.stack(X), np.array(y, dtype='float32')

Xtr, ytr = build_xy_h(df_hybrid.loc[:SPLIT_DATE])
Xte, yte = build_xy_h(df_hybrid.loc[SPLIT_DATE:])

# 数据加载器
train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(
        torch.tensor(Xtr), torch.tensor(ytr).unsqueeze(1)
    ), batch_size=BATCH, shuffle=True)

# 网络同前，只需改输入维度
modelH = LSTM_QR(len(FEATURES_H)).to(DEVICE)
optimH = torch.optim.Adam(modelH.parameters(), lr=1e-3)

# ---------- 训练 (复用早停循环) ----------
# 把前面的训练循环变量名改成 modelH / optimH / BEST_PATH_H
BEST_PATH_H = f"{MODEL_DIR}/lstm_garch_feature.pt"
best_val = np.inf; no_imp = 0
for ep in range(EPOCHS):
    modelH.train(); run=0
    for xb,yb in train_loader:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        loss = quantile_loss(modelH(xb), yb)
        optimH.zero_grad(); loss.backward(); optimH.step()
        run += loss.item()*len(xb)
    tr_loss = run/len(Xtr)
    modelH.eval()
    with torch.no_grad():
        val_loss = quantile_loss(
            modelH(torch.tensor(Xte).to(DEVICE)).cpu(),
            torch.tensor(yte).unsqueeze(1)).item()
    print(f"[H] Ep{ep+1:02d} train{tr_loss:.4f} val{val_loss:.4f}")
    if val_loss < best_val-1e-4:
        best_val = val_loss; no_imp=0
        torch.save(modelH.state_dict(), BEST_PATH_H)
        print("  save best")
    else:
        no_imp+=1
        if no_imp>=patience: print("Early stop."); break


In [ ]:
#步骤 1-c 推断 & 回测
modelH.load_state_dict(torch.load(BEST_PATH_H))
modelH.eval()
with torch.no_grad():
    predH = modelH(torch.tensor(Xte).to(DEVICE)).cpu().numpy().flatten()
VaR_hybrid_feat = pd.Series(-predH, index=df_hybrid.loc[SPLIT_DATE:].index[SEQ_LEN:],
                            name='VaR_hybrid_feat')


# Option 2: Weighted Integration VaR

In [ ]:
#步骤 2-a 准备同日期索引
common_idx = VaR_garch.index.intersection(VaR_lstm.index).intersection(VaR_hs.index)
vg, vl, vh = VaR_garch.loc[common_idx], VaR_lstm.loc[common_idx], VaR_hs.loc[common_idx]
ret_c      = (df_scaled['return']*100).loc[common_idx]


In [ ]:
#步骤 2-b 优化权重
from scipy.optimize import minimize

def combine_var(w, vg, vl, vh):
    """weights = [wg, wl, wh], sum<=1, non-neg"""
    return w[0]*vg + w[1]*vl + w[2]*vh

def objective(w):
    var = combine_var(w, vg, vl, vh)
    return qloss(ret_c, var*100)      # minimize QLoss

# constraints: weights >=0, sum<=1
cons = ({'type':'ineq', 'fun': lambda w: 1 - np.sum(w)},
        {'type':'ineq', 'fun': lambda w: w})
w0 = np.array([0.3,0.4,0.3])
res = minimize(objective, w0, constraints=cons, method='SLSQP')
w_opt = res.x / res.x.sum()            # normalize to sum=1

print("Optimal weights  GARCH | LSTM | HS =", w_opt.round(3))
VaR_ens = combine_var(w_opt, vg, vl, vh).rename('VaR_ensemble')


In [ ]:
#步骤 2-c 回测
b, p = kupiec(ret_c, VaR_ens*100)
print("Ensemble  breaches=",b,"  Kupiec_p=",p,"  QLoss=",qloss(ret_c, VaR_ens*100))


In [ ]:
# ------------------------------------------------------------------
#  Detect which hybrid series you have  ➜  pick first found
# ------------------------------------------------------------------
hybrid_candidates = ['VaR_hybrid_feat', 'VaR_ens']
hybrid_name, VaR_hybrid = None, None
for cand in hybrid_candidates:
    if cand in globals():
        hybrid_name = cand              # e.g. 'VaR_hybrid_feat'
        VaR_hybrid  = globals()[cand]   # the actual Series
        break

if VaR_hybrid is None:
    raise RuntimeError("❌ No hybrid VaR series found. "
                       "Run Feature-Fusion or Ensemble section first.")

print("Using hybrid series:", hybrid_name,
      "| length =", len(VaR_hybrid))

# ------------------------------------------------------------------
#  Align all series to common date index
# ------------------------------------------------------------------
common_idx = (VaR_garch.index
              .intersection(VaR_fhs.index)
              .intersection(VaR_lstm.index)
              .intersection(VaR_hybrid.index))

comp_df = pd.concat({
    'VaR_garch' : VaR_garch.loc[common_idx],
    'VaR_fhs'   : VaR_fhs.loc[common_idx],
    'VaR_lstm'  : VaR_lstm.loc[common_idx],
    'VaR_hybrid': VaR_hybrid.loc[common_idx]
}, axis=1)

ret_plot = (df_scaled['return']*100).loc[common_idx]   # returns still *100

# ------------------------------------------------------------------
#  Plot returns + four VaR curves
# ------------------------------------------------------------------
import matplotlib.pyplot as plt, os
fig, ax = plt.subplots(figsize=(12,4))

(ret_plot/100).plot(ax=ax, color='grey', alpha=0.4, label='Return')
(-comp_df['VaR_garch']).plot( ax=ax, color='red',    label='GARCH')
(-comp_df['VaR_fhs']).plot(   ax=ax, color='blue',   label='FHS',   alpha=0.7)
(-comp_df['VaR_lstm']).plot(  ax=ax, color='orange', label='LSTM',  alpha=0.8)
(-comp_df['VaR_hybrid']).plot(ax=ax, color='purple', label='Hybrid',alpha=0.9)

ax.axhline(0, color='k', lw=0.6)
ax.set_ylabel("Loss level (+) / Return (−)")
ax.set_title("VaR95 — Hybrid vs Baselines")
ax.legend(loc='upper right')

# ------------------------------------------------------------------
#  Save figure
# ------------------------------------------------------------------
SAVE_DIR = "/content/drive/MyDrive/ai-var/models/VAR"
os.makedirs(SAVE_DIR, exist_ok=True)
fig_path = f"{SAVE_DIR}/var_hybrid_vs_baselines.png"
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.show()

print("✅ Figure saved to:", fig_path)


# 11 VaR predictions and backtest

In [ ]:
#11-A整合所有 VaR 序列 + 对齐收益
#  1. 组合所有需要评估的 VaR 列
var_dict = {
    "GARCH"   : VaR_garch,          # 已在步骤 8 得到
    "FHS"     : VaR_fhs,            # 步骤 8
    "HS"      : VaR_hs,             # 步骤 8
    "LSTM"    : VaR_lstm,           # 步骤 9
}

# 可选 Hybrid (若变量存在)
if 'VaR_hybrid_feat' in globals():
    var_dict["HybridFeat"] = VaR_hybrid_feat
if 'VaR_ens' in globals():
    var_dict["HybridEns"]  = VaR_ens

#  2. 统一索引
common_idx = set.intersection(*[set(v.index) for v in var_dict.values()])
common_idx = sorted(common_idx)          # list of dates

var_df  = pd.concat({k: v.loc[common_idx] for k,v in var_dict.items()}, axis=1)
ret_all = (df_scaled['return']*100).loc[common_idx]   # returns still *100
print("Common index length:", len(common_idx))


In [ ]:
#11-B 核心回测函数
from scipy import stats
def kupiec_p(returns, var, alpha=0.05):
    exc   = (returns < -var).astype(int)
    n, x  = len(exc), exc.sum()
    p_hat = x/n
    LR_uc = -2*( np.log((1-alpha)**(n-x) * alpha**x) -
                 np.log((1-p_hat)**(n-x) * p_hat**x) )
    p     = 1-stats.chi2.cdf(LR_uc,1)
    return x, p

def christoffersen_p(returns, var, alpha=0.05):
    exc = (returns < -var).astype(int)
    N00=N01=N10=N11=0
    for i in range(1,len(exc)):
        a,b = exc[i-1], exc[i]
        if a==0 and b==0: N00+=1
        if a==0 and b==1: N01+=1
        if a==1 and b==0: N10+=1
        if a==1 and b==1: N11+=1
    pi0 = N01/(N00+N01) if N00+N01 else 0
    pi1 = N11/(N10+N11) if N10+N11 else 0
    pi  = (N01+N11)/(N00+N01+N10+N11)
    LR_cc = -2*np.log( ((1-pi)**(N00+N10) * pi**(N01+N11)) /
                       (((1-pi0)**N00) * (pi0**N01) * ((1-pi1)**N10) * (pi1**N11)) )
    p = 1-stats.chi2.cdf(LR_cc,2)
    return p

def qloss(returns, var, alpha=0.05):
    y = returns/100
    q = -var
    e = y - q
    return np.mean(np.maximum(alpha*e, (alpha-1)*e))

def mean_excess(returns, var):
    breach = returns < -var
    if breach.any():
        return (-returns[breach] - var[breach]).mean()/100
    return 0.0


In [ ]:
#11-C 批量计算并保存结果表
alpha = 0.05
rows=[]
for name, series in var_df.items():
    b, p_uc = kupiec_p(ret_all, series*100, alpha)
    p_cc    = christoffersen_p(ret_all, series*100, alpha)
    ql      = qloss(ret_all, series*100, alpha)
    mexcess = mean_excess(ret_all, series*100)
    rows.append([name, b, b/len(series), p_uc, p_cc, ql, mexcess])

res_table = pd.DataFrame(rows, columns=["Model","#Breaches","BreachRate",
                                        "Kupiec_p","Christoff_p","PinballLoss",
                                        "MeanExcess"])

print(res_table.sort_values("PinballLoss"))
res_table.to_csv("/content/drive/MyDrive/ai-var/models/result/var_backtest_summary.csv",
                 index=False)


In [ ]:
#11-D 图形：收益 + 全部 VaR + 异常点
fig, ax = plt.subplots(figsize=(13,5))

# plot returns
(ret_all/100).plot(ax=ax, color='grey', lw=0.8, alpha=0.45, label='Return')

# colors for models
colors = dict(GARCH='red', FHS='blue', HS='green', LSTM='orange',
              HybridFeat='purple', HybridEns='brown')
for name, series in var_df.items():
    (-series).plot(ax=ax, label=name, color=colors.get(name,'black'), alpha=0.8)

# breaches (红点)
for name, series in var_df.items():
    breaches = ret_all < -series*100
    ax.scatter(series.index[breaches], (-series[breaches]),
               color=colors.get(name,'black'), s=12, marker='x')

ax.axhline(0, color='k', lw=0.5)
ax.set_title("VaR95 Forecasts vs Returns with Breaches")
ax.set_ylabel("VaR / Return")
ax.legend(ncol=3, fontsize=9)

save_path = "/content/drive/MyDrive/ai-var/models/result/var_all_models.png"
fig.savefig(save_path, dpi=300, bbox_inches='tight'); plt.show()
print("✅ figure saved to:", save_path)


# 12 Result visualization and comparison

In [ ]:
#12-A　柱状图：覆盖率与目标 α
import pandas as pd, matplotlib.pyplot as plt, os, seaborn as sns

# 如果关掉了内存，重新载入
summary = pd.read_csv("/content/drive/MyDrive/ai-var/models/VAR/var_backtest_summary.csv")

fig, ax = plt.subplots(figsize=(7,4))
sns.barplot(x='Model', y='BreachRate', data=summary, palette='Set2', ax=ax)
ax.axhline(0.05, color='k', ls='--', label='Target 5%')
ax.set_title("Breach Rate vs Target (95% VaR)")
ax.set_ylabel("Breach Rate"); ax.legend()

save_path = "/content/drive/MyDrive/ai-var/models/result/breach_rate_bar.png"
fig.savefig(save_path, dpi=300, bbox_inches='tight'); plt.show()
print("保存:", save_path)


In [ ]:
#12-B　Pinball Loss 对比
fig, ax = plt.subplots(figsize=(7,4))
sns.barplot(x='Model', y='PinballLoss', data=summary, palette='Set3', ax=ax)
ax.set_title("Pinball Loss (lower = better)")
save_path = "/content/drive/MyDrive/ai-var/models/result/pinball_loss_bar.png"
fig.savefig(save_path, dpi=300, bbox_inches='tight'); plt.show()
print("保存:", save_path)


In [ ]:
#12-C　雷达图：多指标综合得分
import numpy as np

metrics = ['BreachRate','PinballLoss','MeanExcess']
# 统一方向：越低越好 → 直接使用
data = summary.set_index('Model')[metrics]

# 归一化到 [0,1]
norm = (data - data.min()) / (data.max()-data.min())

labels = norm.index.tolist()
angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False).tolist()
angles += angles[:1]

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, polar=True)
for model in labels:
    values = norm.loc[model].tolist()
    values += values[:1]     # 闭合
    ax.plot(angles, values, label=model)
    ax.fill(angles, values, alpha=0.1)

ax.set_xticks(angles[:-1]); ax.set_xticklabels(metrics)
ax.set_title("Radar — lower is better (normalized)")
ax.legend(loc='upper right', bbox_to_anchor=(1.25,1.05))
save_path = "/content/drive/MyDrive/ai-var/models/result/radar_metrics.png"
fig.savefig(save_path, dpi=300, bbox_inches='tight'); plt.show()
print("保存:", save_path)


In [ ]:
#12-D　异常点时间热图（谁在何时违约）

breach_df = pd.DataFrame({
    m : (ret_all < -var_df[m]*100).astype(int)
    for m in var_df.columns
})
fig, ax = plt.subplots(figsize=(12,4))
sns.heatmap(breach_df.T, cmap=['white','red'], cbar=False, ax=ax)
ax.set_ylabel("Model"); ax.set_xlabel("Date")
ax.set_title("VaR Breach Heatmap (red = exception)")

save_path = "/content/drive/MyDrive/ai-var/models/VAR/breach_heatmap.png"
fig.savefig(save_path, dpi=300, bbox_inches='tight'); plt.show()
print("保存:", save_path)







# 13 Explainable AI

In [ ]:
!pip install --quiet shap scikit-learn tqdm
import shap, torch, joblib, pandas as pd, numpy as np, matplotlib.pyplot as plt, os
shap.initjs()


In [ ]:
#13-A LSTM 全局特征重要度
#13-A-1 准备数据
df_raw = df_raw.loc[:, ~df_raw.columns.duplicated()].copy()

fit_cols  = list(scaler.feature_names_in_)
print("Scaler expects:", fit_cols)

n_before = df_raw.shape[1]
df_raw = df_raw[fit_cols]              # 只保留 & 正确排序
n_after  = df_raw.shape[1]
print(f"Columns after align: {n_after} (before {n_before})")

X_full  = scaler.transform(df_raw.values)     # .values 同 to_numpy()
X_torch = torch.tensor(X_full, dtype=torch.float32).to(DEVICE)

background = X_torch[np.random.choice(len(X_torch), 500, replace=False)]
print("👍 transform success, shape:", X_full.shape)


In [ ]:
#13-A-2 创建 DeepExplainer

# ===================== 13-A-2  DeepExplainer (CPU 安全版) =====================
import copy, shap, torch, numpy as np, pandas as pd, matplotlib.pyplot as plt, os, seaborn as sns
shap.initjs()

SEQ_LEN       = 60
N_BACKGROUND  = 200      # 背景子样本数
N_EXPLAIN     = 1500     # 解释样本数
SAVE_DIR      = "/content/drive/MyDrive/ai-var/xai"
os.makedirs(SAVE_DIR, exist_ok=True)

# ---------- 1) 构造 3-D 序列 ----------
def build_seq(arr, seq_len=SEQ_LEN):
    return np.stack([arr[i-seq_len:i] for i in range(seq_len, len(arr))], axis=0)

X_seq   = build_seq(X_full)                      # (T-SEQ, 60, feat)
print("X_seq shape:", X_seq.shape)

# ---------- 2) 选背景 & 样本 ----------
idx_bg  = np.random.choice(len(X_seq), N_BACKGROUND, replace=False)
idx_exp = np.arange(min(N_EXPLAIN, len(X_seq)))

X_bg_np  = X_seq[idx_bg]
X_exp_np = X_seq[idx_exp]

# ---------- 3) 把模型 & 数据搬到 CPU ----------
model_cpu      = copy.deepcopy(best_model).cpu()   # 深拷贝防止权重冲突
model_cpu.train()                                  # 必须 train() 让 RNN backward 可用

background_cpu = torch.tensor(X_bg_np,  dtype=torch.float32)
X_exp_cpu      = torch.tensor(X_exp_np, dtype=torch.float32)

# ---------- 4) DeepExplainer ----------

model_cpu.train()
explainer = shap.DeepExplainer(model_cpu, background_cpu)   # 无参数
shap_values = explainer.shap_values(X_exp_cpu,              # 参数放这里
                                    check_additivity=False)

# ----- 后续聚合与可视化保持不变 -----


# ---------- 5) 聚合时间维 & 画柱状图 ----------
shap_global = np.abs(shap_vals).mean(axis=(0,1))        # (feat,)
shap_df     = pd.DataFrame({'Feature': scaler.feature_names_in_,
                            'SHAP': shap_global}).sort_values('SHAP', ascending=False)

fig, ax = plt.subplots(figsize=(7,4))
sns.barplot(x='SHAP', y='Feature', data=shap_df, ax=ax, color='steelblue')
ax.set_title("Global SHAP Importance (LSTM-QR)")
plt.tight_layout()

fig_path = f"{SAVE_DIR}/shap_lstm_bar.png"
fig.savefig(fig_path, dpi=300)
plt.show()
print("✅ SHAP bar saved to:", fig_path)




In [ ]:
# -----------------------------------------------------------
# 1️⃣  计算 SHAP 值  (确保版本正确 / 参数位置正确)
# -----------------------------------------------------------
model_cpu.train()
explainer = shap.DeepExplainer(model_cpu, background_cpu)   # 如果已升级 shap≥0.41，可加 check_additivity=False

# ⬇️ 若版本<0.41，需要把 check_additivity 放到函数调用
shap_values = explainer.shap_values(X_exp_cpu, check_additivity=False)

# shap_values 返回 list；取第 0 个输出
if isinstance(shap_values, list):
    shap_vals = shap_values[0]
else:
    shap_vals = shap_values       # 某些版本直接返回 np.array

print("shap_vals shape:", shap_vals.shape)  # (N_samples, seq_len, n_feat)

# -----------------------------------------------------------
# 2️⃣  聚合时间维度 (平均绝对值)
# -----------------------------------------------------------
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt, os

shap_global = np.abs(shap_vals).mean(axis=(0,1))      # -> (n_feat,)
feat_names  = list(scaler.feature_names_in_)           # 保证顺序对应
shap_df     = pd.DataFrame({'Feature': feat_names,
                            'SHAP': shap_global}).sort_values('SHAP', ascending=False)

# -----------------------------------------------------------
# 3️⃣  绘制柱状图
# -----------------------------------------------------------
fig, ax = plt.subplots(figsize=(7,4))
sns.barplot(x='SHAP', y='Feature', data=shap_df, ax=ax, color='steelblue')
ax.set_title("Global SHAP Importance (LSTM-QR)")
plt.tight_layout()

SAVE_DIR = "/content/drive/MyDrive/ai-var/xai"; os.makedirs(SAVE_DIR, exist_ok=True)
fig_path = f"{SAVE_DIR}/shap_lstm_bar.png"
fig.savefig(fig_path, dpi=300)
plt.show()
print("✅ SHAP bar saved to:", fig_path)


In [ ]:
# ---------- 1) 检查并去掉多余维度 ----------
print("raw shap_vals shape:", shap_vals.shape)   # (N, seq_len, feat, 1)
if shap_vals.ndim == 4 and shap_vals.shape[-1] == 1:
    shap_vals = shap_vals.squeeze(-1)            # -> (N, seq_len, feat)
    print("squeezed shap_vals shape:", shap_vals.shape)

# ---------- 2) 聚合时间维度 (绝对值平均) ----------
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt, os

shap_global = np.abs(shap_vals).mean(axis=(0,1))          # (feat,)
feat_names  = list(scaler.feature_names_in_)               # 确保顺序
assert len(shap_global) == len(feat_names), "dimension mismatch"

shap_df = (pd.DataFrame({'Feature': feat_names,
                         'SHAP': shap_global})
             .sort_values('SHAP', ascending=False))

# ---------- 3) 绘制柱状图 ----------
fig, ax = plt.subplots(figsize=(7,4))
sns.barplot(x='SHAP', y='Feature', data=shap_df, ax=ax, color='steelblue')
ax.set_title("Global SHAP Importance (LSTM-QR)")
plt.tight_layout()

SAVE_DIR = "/content/drive/MyDrive/ai-var/xai"; os.makedirs(SAVE_DIR, exist_ok=True)
fig_path = f"{SAVE_DIR}/shap_lstm_bar.png"
fig.savefig(fig_path, dpi=300)
plt.show()
print("✅ SHAP bar saved to:", fig_path)


In [ ]:
#13-A-3 绘制 beeswarm
shap_bee = shap_vals.mean(axis=1)        # shape (N_samples, n_feat)
X_bee = X_exp_np[:, -1, :]               # (N_samples, n_feat)

fig = plt.figure(figsize=(9,6))
shap.summary_plot(
    shap_bee,
    features=X_bee,
    feature_names=list(scaler.feature_names_in_),
    show=False
)

save_dir = "/content/drive/MyDrive/ai-var/xai"; os.makedirs(save_dir, exist_ok=True)
fig.savefig(f"{save_dir}/shap_lstm_beeswarm.png", dpi=300, bbox_inches='tight')
plt.show()
print("✅ beeswarm saved to:", f"{save_dir}/shap_lstm_beeswarm.png")



In [ ]:
#13-B 单日局部解释 (Waterfall)

import shap, torch, pandas as pd, numpy as np, matplotlib.pyplot as plt, os

sample_date = breach_df.index[breach_df.any(axis=1)][0]     # ✔ axis=1
sample_date = pd.Timestamp(sample_date)                     # 转回 Timestamp
print("Sample date:", sample_date)

idx_df = df_raw.index.get_loc(sample_date)                  # 全数据行号
idx_seq = idx_df - SEQ_LEN                                  # 序列张量行号
assert idx_seq >= 0, "sample_date 早于 SEQ_LEN 天，无法解释"

sample_tensor = X_seq_torch[idx_seq : idx_seq+1]            # shape (1,60,feat)

sv = explainer.shap_values(sample_tensor, check_additivity=False)[0][0]  # (60, feat)

sv_avg = sv.mean(axis=0)                                    # (feat,)
base   = explainer.expected_value[0]

shap_exp = shap.Explanation(values=sv_avg,
                            base_values=base,
                            feature_names=scaler.feature_names_in_)

fig = plt.figure(figsize=(8,4))
shap.plots.waterfall(shap_exp, max_display=12, show=False)

save_dir = "/content/drive/MyDrive/ai-var/xai"; os.makedirs(save_dir, exist_ok=True)
fig_path = f"{save_dir}/shap_lstm_{sample_date.strftime('%Y%m%d')}.png"
fig.savefig(fig_path, dpi=300, bbox_inches='tight')
plt.show()

print("✅ Waterfall saved to:", fig_path)



In [ ]:
#13-C Permutation Importance
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt, os, tqdm
from sklearn.metrics import mean_pinball_loss

# ---------- 配置 ----------
SEQ_LEN    = 60
ALPHA      = 0.05                     # 95% VaR 的 pinball α
SAVE_DIR   = "/content/drive/MyDrive/ai-var/xai"
os.makedirs(SAVE_DIR, exist_ok=True)

# ---------- 1) 构造 3-D 序列 ----------
def build_seq(arr, seq_len=SEQ_LEN):
    return np.stack([arr[i-seq_len:i] for i in range(seq_len, len(arr))], axis=0)

X_seq_np = build_seq(X_full)                           # (N_seq, 60, feat)
print("seq shape:", X_seq_np.shape)

# ---------- 2) 目标向量 (确保 1-D) ----------
y_seq_np = df_raw['return'].to_numpy()[SEQ_LEN:]       # 可能 1-D 或 2-D
if y_seq_np.ndim > 1:
    print("⚠️ y_seq_np is 2-D", y_seq_np.shape, "→ take first column")
    y_seq_np = y_seq_np[:, 0]
print("y_seq_np shape:", y_seq_np.shape)               # (N,)

# ---------- 3) 预测函数 (CPU) ----------
model_cpu.eval()                                       # inference 模式
def pred_fn(arr3d):
    with torch.no_grad():
        t = torch.tensor(arr3d, dtype=torch.float32)
        return model_cpu(t).cpu().numpy().flatten()

baseline_pred  = pred_fn(X_seq_np)
baseline_loss  = mean_pinball_loss(y_seq_np, baseline_pred, alpha=ALPHA)
print("baseline pinball loss:", baseline_loss)

# ---------- 4) 手动 permutation importance ----------
feat_names  = list(scaler.feature_names_in_)
importances = []

for j, name in enumerate(tqdm.tqdm(feat_names, desc="permuting")):
    X_perm = X_seq_np.copy()
    # 对每个时间步独立打乱第 j 个特征列，保持序列结构
    for t in range(SEQ_LEN):
        np.random.shuffle(X_perm[:, t, j])
    perm_pred = pred_fn(X_perm)
    loss      = mean_pinball_loss(y_seq_np, perm_pred, alpha=ALPHA)
    importances.append(loss - baseline_loss)           # ↑loss ⇒ 该特征重要

imp_df = (pd.DataFrame({'Feature': feat_names,
                        'Importance': importances})
            .sort_values('Importance', ascending=False))

# ---------- 5) 绘图 & 保存 ----------
fig, ax = plt.subplots(figsize=(7,4))
sns.barplot(x='Importance', y='Feature', data=imp_df,
            ax=ax, color='indianred')
ax.set_title("Permutation Importance (LSTM-QR)")
plt.tight_layout()

fig_path = f"{SAVE_DIR}/perm_importance_bar.png"
fig.savefig(fig_path, dpi=300)
imp_df.to_csv(f"{SAVE_DIR}/perm_importance.csv", index=False)
plt.show()

print("✅ permutation bar & csv saved in:", SAVE_DIR)




In [ ]:
#13-D GARCH 经济解释
params = garch.params[['omega','alpha[1]','beta[1]','nu']]
params.to_frame("Estimate").to_csv(f"{save_dir}/garch_param_table.csv")
print("GARCH params:\n", params)

# 可选：σ_t 与 情绪皮尔逊
garch_sigma_full = pd.concat([sigma_train/100, sigma_test/100])
corr_sent_sigma = garch_sigma_full.corr(df_scaled['sent_bank_7d'])
print("Corr(σ, sentiment) =", corr_sent_sigma)


# 14 Model Save

In [ ]:
#14-A 保存 全部模型权重与预处理对象
import torch, joblib, os, pickle, json, datetime, zipfile

ROOT_DIR   = "/content/drive/MyDrive/ai-var"
MODEL_DIR  = f"{ROOT_DIR}/models"
SCALER_BIN = f"{MODEL_DIR}/feature_scaler.bin"

timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M")

# 1) LSTM-QR
torch.save(best_model.state_dict(), f"{MODEL_DIR}/LSTM/lstm_qr_best_{timestamp}.pt")

# 2) Hybrid (如果有)
if 'modelH' in globals():        # Feature-fusion
    torch.save(modelH.state_dict(),
               f"{MODEL_DIR}/LSTM/lstm_hybrid_feat_{timestamp}.pt")

# 3) 保存 StandardScaler（若未保存）
if not os.path.exists(SCALER_BIN):
    joblib.dump(scaler, SCALER_BIN)

# 4) 保存 GARCH 结果对象 (pickled)
with open(f"{MODEL_DIR}/garch_result_{timestamp}.pkl", "wb") as f:
    pickle.dump(garch, f)

print("✅ Models & scaler saved with timestamp:", timestamp)


In [ ]:
#14-B 保存 全部 DataFrame / VaR 预测结果
import pandas as pd

# 1) 预测序列：
var_df.to_parquet(f"{ROOT_DIR}/processed/var_predictions_{timestamp}.parquet")

# 2) 回测总结表：
res_table.to_csv(f"{ROOT_DIR}/processed/var_backtest_summary_{timestamp}.csv", index=False)

# 3) 加  XAI 重要度表：
imp_df.to_csv(f"{ROOT_DIR}/xai/perm_importance_{timestamp}.csv", index=False)
shap_df.to_csv(f"{ROOT_DIR}/xai/shap_global_{timestamp}.csv", index=False)

print("✅ VaR predictions & metrics saved")


In [ ]:
#14-C 打包 主要工件 便于下载 / 归档
zip_path = f"{ROOT_DIR}/ai_var_artifacts_{timestamp}.zip"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    # —— 模型 —— #
    zf.write(f"{MODEL_DIR}/LSTM/lstm_qr_best_{timestamp}.pt",
             arcname=f"models/lstm_qr_best_{timestamp}.pt")
    if 'modelH' in globals():
        zf.write(f"{MODEL_DIR}/LSTM/lstm_hybrid_feat_{timestamp}.pt",
                 arcname=f"models/lstm_hybrid_feat_{timestamp}.pt")
    zf.write(SCALER_BIN, "models/feature_scaler.bin")
    zf.write(f"{MODEL_DIR}/garch_result_{timestamp}.pkl",
             arcname=f"models/garch_result_{timestamp}.pkl")

    # —— 数据 & 指标 —— #
    zf.write(f"{ROOT_DIR}/processed/var_predictions_{timestamp}.parquet",
             arcname=f"data/var_predictions_{timestamp}.parquet")
    zf.write(f"{ROOT_DIR}/processed/var_backtest_summary_{timestamp}.csv",
             arcname=f"data/var_backtest_summary_{timestamp}.csv")

    # —— 图表示例 —— #
    zf.write(f"{ROOT_DIR}/models/VAR/var_all_models.png",
             arcname=f"fig/var_all_models.png")
    zf.write(f"{ROOT_DIR}/xai/shap_lstm_bar.png",
             arcname=f"fig/shap_lstm_bar.png")

print("✅ zipped:", zip_path)


In [ ]:
#14-D 将 复现信息 保存成 metadata.json
meta = {
    "created": timestamp,
    "models": {
        "lstm_qr": f"models/LSTM/lstm_qr_best_{timestamp}.pt",
        "hybrid_feat": f"models/LSTM/lstm_hybrid_feat_{timestamp}.pt" if 'modelH' in globals() else None,
        "garch": f"models/garch_result_{timestamp}.pkl"
    },
    "scaler": "models/feature_scaler.bin",
    "var_predictions": f"processed/var_predictions_{timestamp}.parquet",
    "metrics": f"processed/var_backtest_summary_{timestamp}.csv",
    "environment": {
        "python": "3.11",
        "torch": torch.__version__,
        "shap": shap.__version__
    },
    "comment": "AI-driven VaR project – full artifact snapshot"
}

with open(f"{ROOT_DIR}/metadata_{timestamp}.json", "w") as f:
    json.dump(meta, f, indent=2)

print("✅ metadata saved:", f"{ROOT_DIR}/metadata_{timestamp}.json")


In [ ]:
import torch, joblib, pandas as pd, numpy as np

# --- 路径 --- (换成最新 time-stamp)
model_path   = f"{ROOT_DIR}/models/LSTM/lstm_qr_best_{timestamp}.pt"
scaler_path  = f"{ROOT_DIR}/models/feature_scaler.bin"
lstm_loaded  = LSTM_QR(len(FEATURES)).cpu()
lstm_loaded.load_state_dict(torch.load(model_path, map_location='cpu'))
lstm_loaded.eval()
scaler_loaded = joblib.load(scaler_path)

# --- 准备一个新的 60×feat 序列 ---
new_raw   = ...  # 你的新 DataFrame
new_scaled = scaler_loaded.transform(new_raw[FEATURES])
new_seq    = new_scaled[np.newaxis, ...]          # shape (1, 60, feat)
with torch.no_grad():
    pred_q = lstm_loaded(torch.tensor(new_seq, dtype=torch.float32)).item()
VaR95_new = -pred_q
print("Predicted VaR95:", VaR95_new)
